In [190]:
import pandas as pd
import sqlite3
df = pd.read_csv("data_2022_2023.csv")
df


,LastName,FirstName,Gender,Country,Date,Competition,Round,Location,Apparatus,Rank,D_Score,E_Score,Penalty,Score
0,AAS,Fredrik,m,NOR,24-27 Feb 2022,2022 Cottbus World Cup,qual,"Cottbus, Germany",HB,18.0,3.9,8.266,NaN,12.166
1,AAS,Fredrik,m,NOR,24-27 Feb 2022,2022 Cottbus World Cup,qual,"Cottbus, Germany",PB,23.0,3.9,6.900,NaN,10.800
2,AAS,Fredrik,m,NOR,24-27 Feb 2022,2022 Cottbus World Cup,qual,"Cottbus, Germany",PH,33.0,4.2,6.666,NaN,10.866
3,AAS,Fredrik,m,NOR,23-26 Feb 2023,2023 Cottbus World Cup,qual,"Cottbus, Germany",HB,39.0,4.6,6.700,NaN,11.300
4,AAS,Fredrik,m,NOR,23-26 Feb 2023,2023 Cottbus World Cup,qual,"Cottbus, Germany",PH,44.0,4.4,7.800,NaN,12.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24429,ÖNDER,Ahmet,m,TUR,1-4 Jun 2023,2023 Tel Aviv World Challenge Cup,final,"Tel Aviv, Israel",FX,8.0,4.8,7.050,NaN,11.850
24430,ÖNDER,Ahmet,m,TUR,1-4 Jun 2023,2023 Tel Aviv World Challenge Cup,qual,"Tel Aviv, Israel",FX,3.0,5.8,7.950,0.1,13.650
24431,ÖNDER,Ahmet,m,TUR,1-4 Jun 2023,2023 Tel Aviv World Challenge Cup,qual,"Tel Aviv, Israel",HB,10.0,5.0,6.250,NaN,11.250
24432,ÖNDER,Ahmet,m,TUR,1-4 Jun 2023,2023 Tel Aviv World Challenge Cup,final,"Tel Aviv, Israel",PB,1.0,6.3,8.050,NaN,14.350


In [191]:
df.dropna(inplace=True, subset=[ 'Apparatus', 'Score', 'Country', 'D_Score', "E_Score"])
df.loc[df['Apparatus'] == 'VT1', 'Apparatus'] = 'VT'
df.loc[df['Apparatus'] == 'VT2', 'Apparatus'] = 'VT'


In [192]:
with sqlite3.connect("gym") as conn:
    df.to_sql("gym", conn, if_exists = "replace", index = False) 
    

In [193]:
with sqlite3.connect("gym") as conn:
    cmd = \
    """
    SELECT * FROM gym
    """
    sql = pd.read_sql_query(cmd, conn)
    print(sql)

      LastName FirstName Gender Country            Date  \
0          AAS   Fredrik      m     NOR  24-27 Feb 2022   
1          AAS   Fredrik      m     NOR  24-27 Feb 2022   
2          AAS   Fredrik      m     NOR  24-27 Feb 2022   
3          AAS   Fredrik      m     NOR  23-26 Feb 2023   
4          AAS   Fredrik      m     NOR  23-26 Feb 2023   
...        ...       ...    ...     ...             ...   
24429    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   
24430    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   
24431    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   
24432    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   
24433    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   

                             Competition  Round          Location Apparatus  \
0                 2022 Cottbus World Cup   qual  Cottbus, Germany        HB   
1                 2022 Cottbus World Cup   qual  Cottbus, Germany        PB   
2                 2022 Cottbus World Cup   qual  Cottb

In [ ]:
def query_gym_database(country ):
    with sqlite3.connect('gym') as conn:
        cmd = \
        f'''
SELECT LastName, FirstName, Apparatus, AVG(Score) AS PredictedScore, MAX(Score) as Maxscore, 
SQRT(AVG(score * score) - AVG(score) * AVG(score)) AS StdDevScore,
COUNT(DISTINCT Date) AS CompetitionsCount
FROM gym 
WHERE Country = '{country}'
GROUP BY LastName, FirstName, Apparatus 
ORDER BY Apparatus, Maxscore DESC
        '''
    df = pd.read_sql_query(cmd, conn)
    return (df)

df = query_gym_database(country="NOR")

In [208]:
import plotly
from plotly import express as px
import plotly.io as pio

fig = px.scatter(df,
            x = "Apparatus", 
            y = "PredictedScore", 
            color="LastName",
            size='CompetitionsCount', hover_data=['PredictedScore'])


fig.update_layout(
    title=f"Scatterplot of Gymnasts' MaxScore For Different Apparatus", #The colorbar and overall plot have professional titles.
    yaxis_title="Max Score of Gymnasts",
    )


fig.show()